Data Enginee

In [48]:
# Dependancies
import pandas as pd
import os
import csv

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base

# Define Declarative Base 
Base = declarative_base()

In [39]:
# Read in CSV File
df=pd.read_csv('datasets/percentage_by_state.csv', dtype={'Zip': 'str'})
df.head(2)

,state,abr,lat,lng,CancerType,incidence,population,percentage_incident,prevalence,alive,year,death_count,prevalence_1_year,percentage_deaths,diff
0,Alabama,AL,32.6010112,-86.680736,Female Breast,18227,12434320,0.146586,"0,15",1865148,2011-2015,3394,"0,150272954",0.027295,0
1,Alaska,AK,61.3025006,-158.775020,Female Breast,2100,1751388,0.119905,"0,12","210166,56",2011-2015,310,"0,120177254",0.017700,0


In [40]:
# Inspecting the Data
df.shape

(50, 15)

In [41]:
# Drop unnessesary columns. NOTE: data has no Missing/NaN values
dropped_df = df.drop(['CancerType', 'prevalence', 'alive', 'year', 'prevalence_1_year', 'diff'], axis=1)
dropped_df.head(2)

,state,abr,lat,lng,incidence,population,percentage_incident,death_count,percentage_deaths
0,Alabama,AL,32.6010112,-86.680736,18227,12434320,0.146586,3394,0.027295
1,Alaska,AK,61.3025006,-158.775020,2100,1751388,0.119905,310,0.017700


In [42]:
# Creating an engine, sqlite database and connection to the engine
engine = create_engine("sqlite:///usa_percentage_state.sqlite", echo=False)
conn = engine.connect()

In [43]:
# Creating a Class & table 
class Leads(Base):
    __tablename__ = 'usa_percentage_state_table'
    id = Column(Integer, primary_key=True)
    state = Column(String(255))
    abr = Column(String(255))
    lat = Column(Float)
    lng = Column(Float)
    incidence = Column(Float)
    population = Column(Float)
    percentage_incident = Column(Float)
    death_count = Column(Float)
    percentage_deaths = Column(Float)   

In [44]:
Base.metadata.create_all(conn)

In [45]:
# Transferring the data from dataframe to the sqlite db
dropped_df.to_sql(name='usa_percentage_state_table', con=conn, if_exists='append', index=False)

In [46]:
# Inspecting the Table to make sure the table is there
inspector = inspect(engine)
inspector.get_table_names()

['usa_percentage_state_table']

In [47]:
# Checking the data 
engine.execute('SELECT * FROM usa_percentage_state_table LIMIT 10').fetchall()

[(1, 'Alabama', 'AL', 32.6010112, -86.6807365, 18227.0, 12434320.0, 0.14658622300000002, 3394.0, 0.027295421),
 (2, 'Alaska', 'AK', 61.3025006, -158.7750198, 2100.0, 1751388.0, 0.11990489800000001, 310.0, 0.017700247),
 (3, 'Arizona', 'AZ', 34.1682185, -111.930907, 22214.0, 16686987.0, 0.133121695, 3957.0, 0.023713089),
 (4, 'Arkansas', 'AR', 34.7519275, -92.13137840000002, 10399.0, 7527040.0, 0.138155238, 2050.0, 0.027235142),
 (5, 'California', 'CA', 37.2718745, -119.2704153, 128452.0, 96473279.0, 0.13314775, 22033.0, 0.022838447999999997),
 (6, 'Colorado', 'CO', 38.9979339, -105.55056699999999, 17933.0, 13143731.0, 0.136437668, 2796.0, 0.021272499),
 (7, 'Connecticut', 'CT', 41.5187835, -72.75750699999999, 16000.0, 9201326.0, 0.17388798100000002, 2288.0, 0.024865981000000002),
 (8, 'Delaware', 'DE', 39.145251, -75.41892059999999, 3994.0, 2387966.0, 0.16725531300000002, 672.0, 0.028141104),
 (9, 'District of C', 'DC', 38.8993487, -77.0145666, 2350.0, 1700582.0, 0.138187985, 509.0, 0.